## Sentence-Window retrieval RAG(v1.2) improve the retriever by re-ranking
1. Forcus on Individual sentences 
2. Search the query embedding with most relevant sentence in the pdf 
3. Perform top k similarity with the sentence and retrieve the relevant sentence with surrounding sentences as context.

### Import relevant libraries from llama-Index

In [21]:
# basic modules
import os
import re
from pprint import pprint
import json

# llama index for rag system
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import load_index_from_storage
from llama_index.core import Document
from llama_index.core import Settings
from llama_index.core import StorageContext
from llama_index.core import ChatPromptTemplate
from llama_index.core import PromptTemplate

# ollama model with llama index
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

# evaluation metrics
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

In [2]:
documents = SimpleDirectoryReader(
    input_files=["./data/About us.pdf", 
                 "./data/contact us.pdf",
                 "./data/delivery.pdf",
                 "./data/payment options.pdf",
                 "./data/returns.pdf",
                 "./data/services.pdf",
                 "./data/testings.pdf",
                 "./data/tracking.pdf",
                 "./data/warranty.pdf",
                 "./data/whatsapp order.pdf"]
).load_data()

print(len(documents), "\n")
print(type(documents[0]))

18 

<class 'llama_index.core.schema.Document'>


In [3]:
# initial examine the text in the documents
for doc in documents:
    pprint(doc.text[:100])

('About Us  \n'
 'TRONIC.LK is an electronic component and module sourcing company under Sigma '
 'Electronics ')
('Contact us \n'
 'Electronic Shop - Kohuwala, Nugegoda  \n'
 'TRONIC.LK  \n'
 '8, 1/1, Sunethradevi Road,  \n'
 'Kohuwala')
("If the item(s) (components or modules) you need isn't listed, we can try to "
 'supply that for you. \n'
 'Yo')
('Delivery \n'
 'DELIVERY VIA COURIER SERVICE  \n'
 ' \n'
 'We deliver items island-wide via a courier service. The c')
('If you have any question regarding delivery via courier service, please '
 'contact us via \n'
 'info@tronic.')
('Payment Options \n'
 'Here are the payment options we offer. \n'
 'We prefer bank transfers to all other optio')
('Returns \n'
 'We reserve the right to accept or reject return or exchange requests other '
 'than Warranty \n'
 'C')
('Services \n'
 'As most of you know, TRONIC.LK is running by set of engineers who have years '
 'of experience')
('If you need to mould your enclosure or any other part using pla

In [4]:
# there are \n in the document so clean the text of the documents
def clean_text_fn(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

# save cleaned text of each document in an array
clean_texts = [clean_text_fn(document.text) for document in documents]

In [5]:
# load data function has divided the document in to pages and made document objects from them
# including texts
for doc in documents:
    print(doc.metadata)

# concatenate data to one document and separate each pdf with \n\n
document = Document(text="\n\n".join([txt for txt in clean_texts]))
print(document.text[:1000])

{'page_label': '1', 'file_name': 'About us.pdf', 'file_path': 'data\\About us.pdf', 'file_type': 'application/pdf', 'file_size': 88750, 'creation_date': '2025-02-28', 'last_modified_date': '2025-02-28'}
{'page_label': '1', 'file_name': 'contact us.pdf', 'file_path': 'data\\contact us.pdf', 'file_type': 'application/pdf', 'file_size': 104443, 'creation_date': '2025-02-28', 'last_modified_date': '2025-02-28'}
{'page_label': '2', 'file_name': 'contact us.pdf', 'file_path': 'data\\contact us.pdf', 'file_type': 'application/pdf', 'file_size': 104443, 'creation_date': '2025-02-28', 'last_modified_date': '2025-02-28'}
{'page_label': '1', 'file_name': 'delivery.pdf', 'file_path': 'data\\delivery.pdf', 'file_type': 'application/pdf', 'file_size': 103930, 'creation_date': '2025-02-28', 'last_modified_date': '2025-02-28'}
{'page_label': '2', 'file_name': 'delivery.pdf', 'file_path': 'data\\delivery.pdf', 'file_type': 'application/pdf', 'file_size': 103930, 'creation_date': '2025-02-28', 'last_mod

### Change the window size to 1

In [6]:
# node parser get the sentences from the document and create node based on sentences and include the surrounding context
node_parser = SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=2,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)

In [7]:
llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1,request_timeout=120)

response = llm.complete("Who is Laurie Voss? write in 10 words")

In [8]:
print(response)

<think>
Okay, so I need to figure out who Laurie Voss is and how she got the nickname "Laurie Voss." Let me start by recalling what I know about her. She's a former U.S. Vice President, right? So, she was someone important in politics.

I remember that nicknames can be derived from initials or other personal details. Laurie Voss probably started with her first name and then maybe the last name. Maybe "Laurie" comes from her first name, and "Voss" is a play on her last name? Or perhaps it's a combination of both.

Wait, I think she was known for her political career, especially in the 1980s. She ran for office in several states, so maybe that's where her nickname came from. Also, sometimes nicknames are given based on how people refer to them or their achievements.

I should check if there's a more specific reason behind "Laurie Voss." Maybe it's because she was the wife of a famous person? I'm not sure about that. Alternatively, it could be a nickname used by her campaign team or frien

### Setup embedding model from the llama-Index

1. we can set up services globally by Settings

- Settings.llm = OpenAI(model="gpt-3.5-turbo")
- Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
- Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
- Settings.num_output = 512
- Settings.context_window = 3900

2. we can set up locally 

-     sentence_index = VectorStoreIndex.from_documents(
        [document],
        node_parser=node_parser,
        embed_model=ollama_embedding)

In [13]:
# make service context to rag application 
# nomic-embed-text:latest as embedding model to the service context
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

# a vector store index only needs an embed model
# base node parser is a sentence splitter
text_splitter = SentenceSplitter()


### Extract Nodes

you can extract the nodes using node parser this gives the nodes that are with context and the base sentence.

you can extract the base sentence by the normal parser sentencesplitter().

In [11]:
# extract nodes and the sentences context using node parser 
nodes = node_parser.get_nodes_from_documents([document])
# extract sentences using sentenceSplitter()
base_nodes = text_splitter.get_nodes_from_documents([document])
print(len(base_nodes))
print(len(nodes))
print(nodes[1].metadata)

7
201
{'window': 'About Us TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd.  We supply high quality electronic components, modules and tools manufactured in China, Taiwan, Hong Kong, Japan, USA, Italy, England, Germany and Australia.  Being electronic hobbyists, we strive to provide high-quality modules and components at a reasonable price.  We currently supply Arduinos, Raspberry Pis, Orange Pis, Micro:bits, NodeMCUs, Atmel & Microchip microcontrollers, ICs & other passive components, Creality 3D Printers & filaments, Pneumatic parts, CNC accessories, Inverters, Battery chargers, Multimeters, Ronix Tools, etc... We were the first to introduce Arduino development boards and modules to Sri Lanka way back in 2011 under the company named Lankatronics (Pvt) Ltd., which later evolved into TRONIC.LK. ', 'original_sentence': 'We supply high quality electronic components, modules and tools manufactured in China, Taiwan, Hong Kong, Japan, USA, 

above mentioned way splitting the sentences are not well fit to the given scenario. Try of the regular expression custom splitting.

In [9]:
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s+', document.text.strip())
print(sentences)

['About Us TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd.', 'We supply high quality electronic components, modules and tools manufactured in China, Taiwan, Hong Kong, Japan, USA, Italy, England, Germany and Australia.', 'Being electronic hobbyists, we strive to provide high-quality modules and components at a reasonable price.', 'We currently supply Arduinos, Raspberry Pis, Orange Pis, Micro:bits, NodeMCUs, Atmel & Microchip microcontrollers, ICs & other passive components, Creality 3D Printers & filaments, Pneumatic parts, CNC accessories, Inverters, Battery chargers, Multimeters, Ronix Tools, etc...', 'We were the first to introduce Arduino development boards and modules to Sri Lanka way back in 2011 under the company named Lankatronics (Pvt) Ltd., which later evolved into TRONIC.LK.', 'Today, TRONIC.LK is one of the leading electronic stores in Sri Lanka.', 'In our clientele, we have all the leading universities, technology instit

In [10]:
def custom_sentence_splitter(text):
    # Apply the regex pattern to split sentences
    sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s+", text)
    return sentences

node_parser = SentenceWindowNodeParser.from_defaults(
    # to split the sentences us a custom splitter
    sentence_splitter=custom_sentence_splitter,
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)

In [11]:
# from the whole document with all details of the company
nodes = node_parser.get_nodes_from_documents([document])
for node in nodes[:2]:
    print(node.metadata['original_sentence'], sep='/n')
    pprint(node.metadata) 

About Us TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd.
{'original_sentence': 'About Us TRONIC.LK is an electronic component and '
                      'module sourcing company under Sigma Electronics (Pvt) '
                      'Ltd.',
 'window': 'About Us TRONIC.LK is an electronic component and module sourcing '
           'company under Sigma Electronics (Pvt) Ltd. We supply high quality '
           'electronic components, modules and tools manufactured in China, '
           'Taiwan, Hong Kong, Japan, USA, Italy, England, Germany and '
           'Australia. Being electronic hobbyists, we strive to provide '
           'high-quality modules and components at a reasonable price. We '
           'currently supply Arduinos, Raspberry Pis, Orange Pis, Micro:bits, '
           'NodeMCUs, Atmel & Microchip microcontrollers, ICs & other passive '
           'components, Creality 3D Printers & filaments, Pneumatic parts, CNC '
     

In [14]:
# index the documents based on the document and sentence splitting
# if the current ./sentence_index is already created no need to again create the indexes
if not os.path.exists("./sentence_index"):
    # build the index
    sentence_index = VectorStoreIndex(nodes,embed_model=ollama_embedding)
    # store it in persistent document
    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    # if available load them back
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        embed_model=ollama_embedding
    )

In [15]:
# check whether the given sentence index are in correct form
docstore = sentence_index.storage_context.docstore

# get the nodes data
for node_id, node in docstore.docs.items():
    print(f"Node ID: {node_id}")
    print(f"Content: {node.get_content()}")
    print()

Node ID: d3326a5b-61f5-4387-843e-5c713e591edc
Content: About Us TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd.

Node ID: 7c640d46-30bb-4653-96f6-d07bbea7e1ac
Content: We supply high quality electronic components, modules and tools manufactured in China, Taiwan, Hong Kong, Japan, USA, Italy, England, Germany and Australia.

Node ID: 029415d0-a6f7-4a8d-ab03-37586feef686
Content: Being electronic hobbyists, we strive to provide high-quality modules and components at a reasonable price.

Node ID: aa6323ce-3249-4f92-a698-7066de1a4d9d
Content: We currently supply Arduinos, Raspberry Pis, Orange Pis, Micro:bits, NodeMCUs, Atmel & Microchip microcontrollers, ICs & other passive components, Creality 3D Printers & filaments, Pneumatic parts, CNC accessories, Inverters, Battery chargers, Multimeters, Ronix Tools, etc...

Node ID: a61637ae-4a62-4df7-862c-607208f42a69
Content: We were the first to introduce Arduino development boards and modules 

In [24]:
# simple query engine to retrieve the most relevant context to the given question
query_engine = sentence_index.as_query_engine(
    # query engine use the llm as deepseek previous mentioned model
    llm=llm,
    # only get the most relevant
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    # node parser has the window key in metadata that with the sentence with surrounding context
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

# query function get the llm and the retrieve sentence window 
window_response = query_engine.query(
    "How to contact them?"
)
print(window_response)

<think>
Okay, so I need to figure out how to contact Lankatronics based on the given context. Let me read through the context again carefully.

First, there's a section about releasing more resources near future. It mentions that we'll take over some orders for moulding and PCB production in collaboration with Chinese partners. They want initial communication via email. So if someone wants to reach out, they might send an email to projects@lankatronics.lk. I should make sure not to mention any prior knowledge or statements like "Based on the context..." because that's against the rules.

Next, there's a section about 3D Designing. It says we have an expert in-house engineer for 3D modeling and design work. If someone needs help designing an enclosure for their electronic product, they can reach out to this engineer. So if I need assistance with the design part, I should contact them directly.

Then there's a Battery Chargers section. It mentions that if you only have less than five ite

In [ ]:
pprint(window_response)

Response(response='<think>\n'
                  'Okay, so I need to figure out how to contact Lankatronics '
                  'based on the provided context. Let me read through the '
                  'context carefully.\n'
                  '\n'
                  "First, there's a section about releasing more resources "
                  'near future. It mentions that initial communication will be '
                  'via email. So if someone wants to reach them directly, they '
                  'might send an email. The context says "send your inquiry to '
                  'projects@lankatronics.lk with your contact details." That '
                  'sounds like the standard way to contact them.\n'
                  '\n'
                  "Then there's a battery chargers section. It talks about "
                  'WhatsApp as an alternative method for ordering when you '
                  'have fewer items. So if someone prefers using WhatsApp, '
                  "that could 

In [19]:
pprint(query_engine.get_prompts())

{'response_synthesizer:refine_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.REFINE: 'refine'>}, template_vars=['query_str', 'existing_answer', 'context_msg'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.REFINE: 'refine'>}, template_vars=['query_str', 'existing_answer', 'context_msg'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="The original query is as follows: {query_str}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\nRefined Answer: "), conditionals=[(<function is_chat_model at 0x0000020E5CDE23E0>, ChatPromptTemplate(metada

### Update the prompt template for more precise and polite answer

In [25]:
# customer service chatbot
qa_prompt_tmpl_str = (
    "You are a helpful and professional customer service chatbot. Your job is to provide accurate, "
    "concise, and friendly responses to customer inquiries based on the provided context.\n"
    "---------------------\n"
    "Context Information:\n"
    "{context_str}\n"
    "---------------------\n"
    "Please ensure your response is relevant, clear, and aligns with company policies.\n"
    "If the context does not provide sufficient information, politely ask for clarification.\n"
    "Do not make assumptions beyond the given context.\n"
    "Use a professional yet friendly tone.\n\n"
    "Customer Query: {query_str}\n"
    "Response: "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [23]:
# test run based on above chat template
questions_answers = [
  {
    "question": "What is TRONIC.LK?",
    "expected_answer": "TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd."
  }
]

for question in questions_answers:
    # input question 
    input_question = question["question"]
    # query using query engine
    actual_output = query_engine.query(input_question)
    # print the output of the model
    print(actual_output)

<think>
Okay, so I need to figure out what TRONIC.LK is. From the context provided, it seems like TRONIC.LK is a company that's part of Sigma Electronics and operates as a sourcing company for electronic components and modules. They supply various products from different countries like China, Taiwan, Hong Kong, Japan, etc.

The user asked, "What is TRONIC.LK?" I should provide a clear and concise answer based on the information given. The context mentions that they're an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd., so I can start by stating that.

I also need to mention their range of products, such as Arduinos, Raspberry Pis, Orange Pis, microcontrollers, ICs, Creality 3D Printers, etc. It's important to highlight that they're electronic hobbyists and aim for high-quality components at reasonable prices.

Additionally, the context notes that they were the first to introduce Arduino development boards in Sri Lanka in 2011 and are now leading a lo

### (v2.1)Do the evaluation based on experimental setup

model:"deepseek-r1:1.5b"

window_size:2

prompt template updated

sentence_window_retrieval

In [26]:
# evaluation dataset 
questions_answers = [
  {
    "question": "What is TRONIC.LK?",
    "expected_answer": "TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd."
  },
  {
    "question": "What types of products does TRONIC.LK supply?",
    "expected_answer": "They supply high quality electronic components, modules, and tools manufactured in China, Taiwan, Hong Kong, Japan, USA, Italy, England, Germany, and Australia."
  },
  {
    "question": "When did TRONIC.LK first introduce Arduino development boards and modules to Sri Lanka?",
    "expected_answer": "They introduced them back in 2011 under the company named Lankatronics (Pvt) Ltd."
  },
  {
    "question": "What is the mission of TRONIC.LK?",
    "expected_answer": "To introduce high quality components to the local market."
  },
  {
    "question": "What is the vision of TRONIC.LK?",
    "expected_answer": "To become the market leader in high quality component sourcing."
  },
  {
    "question": "Where is the TRONIC.LK Electronic Shop located?",
    "expected_answer": "At 8, 1/1, Sunethradevi Road, Kohuwala, Nugegoda."
  },
  {
    "question": "What are the operating hours of the TRONIC.LK Electronic Shop?",
    "expected_answer": "Open all 7 days from 9:00 AM to 7:00 PM, except on Poya Days and Mercantile Holidays."
  },
  {
    "question": "How can customers check product availability at TRONIC.LK?",
    "expected_answer": "They should send an email to check@tronic.lk."
  },
  {
    "question": "What telephone numbers are provided for general inquiries at the Electronic Shop?",
    "expected_answer": "0777 854444 and 0114 544566 (Tel/WhatsApp)."
  },
  {
    "question": "What are the operating hours for the TRONIC.LK Online Store?",
    "expected_answer": "It is open from Monday to Saturday, 9:00 AM to 6:00 PM, except on Poya Days and Mercantile Holidays."
  },
  {
    "question": "Which email address should be used for Sales Inquiries?",
    "expected_answer": "sales@tronic.lk"
  },
  {
    "question": "What is the procedure for special orders?",
    "expected_answer": "Customers need to send an email to orders@tronic.lk with their contact details and details of the required items including website links."
  },
  {
    "question": "How does TRONIC.LK provide technical support?",
    "expected_answer": "Technical support is provided via their Support Ticket System; inquiries should be sent to support@tronic.lk."
  },
  {
    "question": "Which email address is designated for Corporate Division inquiries?",
    "expected_answer": "corp@lankatronics.lk"
  },
  {
    "question": "What is the Corporate Division hotline number?",
    "expected_answer": "077 5678 000 (Tel & WhatsApp)."
  },
  {
    "question": "Which division handles R&D and what is their contact email?",
    "expected_answer": "Tesla Robotics (Pvt) Ltd. handles R&D; contact them at projects@tesla.lk."
  },
  {
    "question": "How does TRONIC.LK deliver orders across the island?",
    "expected_answer": "They deliver items via a courier service with charges based on the weight of the parcel."
  },
  {
    "question": "What is the courier charge for the first kilogram?",
    "expected_answer": "Rs. 480 for the first kilogram."
  },
  {
    "question": "How much is charged for each additional kilogram in courier delivery?",
    "expected_answer": "Rs. 150 for each additional kilogram."
  },
  {
    "question": "What is the delivery guarantee period for courier services?",
    "expected_answer": "Delivery is guaranteed within 1 to 4 working days."
  },
  {
    "question": "How do customers track their parcel?",
    "expected_answer": "A tracking number is provided via email, and customers can track their parcel through the courier service's tracking system."
  },
  {
    "question": "What quick delivery option is available for customers in Colombo?",
    "expected_answer": "Quick Delivery via Uber or Pickme Flash motorbike/three-wheeler delivery service."
  },
  {
    "question": "How should a customer initiate a fast delivery request using Uber/Pickme?",
    "expected_answer": "Place an online order and enter 'Uber / Pickme collection' in the comments box; then follow the provided instructions."
  },
  {
    "question": "What does a customer need to do for a store pickup order?",
    "expected_answer": "They should mention 'Store Pickup' in the comment section when placing the order."
  },
  {
    "question": "What is the preferred payment method at TRONIC.LK?",
    "expected_answer": "Bank transfers are preferred due to the absence of unnecessary third-party charges."
  },
  {
    "question": "What mobile app can be used for payments that link to multiple bank accounts?",
    "expected_answer": "The Frimi app by Nations Trust Bank (NTB)."
  },
  {
    "question": "What processing fee is applied when using a QR payment app like Frimi QR?",
    "expected_answer": "A processing fee of 0.5% is applied."
  },
  {
    "question": "What processing fee is charged when using PayHere for card payments?",
    "expected_answer": "A processing fee of 2.99% is charged."
  },
  {
    "question": "How can customers opt for easy installment payments?",
    "expected_answer": "By requesting an online payment request via PayHere which offers several easy payment options."
  },
  {
    "question": "What is TRONIC.LK’s policy regarding returns and exchanges?",
    "expected_answer": "Return/exchange requests (other than warranty claims) are accepted only if items are unopened, sealed, and unused, usually within 7 days of purchase."
  },
  {
    "question": "What is the return procedure for items purchased from the physical store?",
    "expected_answer": "Customers must visit the outlet with their bill and the item(s) to be exchanged; the store will take their details and inform them within 7 days."
  },
  {
    "question": "How should online purchase returns be initiated?",
    "expected_answer": "By sending an email to info@tronic.lk with contact details, bill date/number, and a list of items to be returned. Additional instructions may follow."
  },
  {
    "question": "Apart from selling products, what additional services does TRONIC.LK offer?",
    "expected_answer": "They offer services such as 3D printing, CNC PCB engraving, 3D designing, PCB designing, PCB engraving/etching, PCB production, firmware development, and testing services."
  },
  {
    "question": "How does TRONIC.LK support local technology and innovation?",
    "expected_answer": "They share resources like 3D printers and CNC PCB engravers for prototyping purposes at reasonable costs and facilitate limited mass-production orders."
  },
  {
    "question": "What service is available for customers needing 3D design?",
    "expected_answer": "In-house 3D modeling and designing for creating enclosures or prototypes, available by contacting projects@lankatronics.lk."
  },
  {
    "question": "How do customers inquire about 3D printing services?",
    "expected_answer": "They should send their STL file to projects@lankatronics.lk for a quotation."
  },
  {
    "question": "What is the minimum order quantity for plastic injection moulding?",
    "expected_answer": "The minimum order quantity is 1000 pieces."
  },
  {
    "question": "How does TRONIC.LK assist with PCB designing?",
    "expected_answer": "They provide in-house PCB designing services; customers can send their project details to projects@lankatronics.lk."
  },
  {
    "question": "What should a customer do to request firmware development services?",
    "expected_answer": "They need to send a detailed inquiry with their requirements to projects@lankatronics.lk."
  },
  {
    "question": "What testing services does TRONIC.LK offer for modules and sensors?",
    "expected_answer": "They perform tests on modules and sensors, providing reports with photos, videos, diagrams, and source codes, with the cost paid in advance."
  },
  {
    "question": "What happens if a module sold by TRONIC.LK is found to be malfunctioning due to a manufacturing defect?",
    "expected_answer": "TRONIC.LK will either offer a refund or a replacement free of charge."
  },
  {
    "question": "What are some conditions that may void the warranty provided by TRONIC.LK?",
    "expected_answer": "Warranty may be voided by factors such as burn marks, mechanical damage during transportation, unauthorized modifications, inappropriate installation, misuse, external events, and non-observance of documentation."
  },
  {
    "question": "What special warranty terms apply to inverters and battery chargers?",
    "expected_answer": "For inverters, grid power should not be connected to the output and usage should not exceed half the marked output peak wattage. For battery chargers, batteries must be disconnected during charging and continuous charging of weak batteries is not advised."
  }
]

- "The RAG triad is composed of three RAG evaluation metrics: answer relevancy, faithfulness, and contextual relevancy. If a RAG pipeline scores high on all three metrics, we can confidently say that our RAG pipeline is using the optimal hyperparameters."
    * Answer relevancy: the answer relevancy metric determines how relevant the answers generated by your RAG generator is. Since LLMs nowadays are getting pretty good at reasoning, it is mainly the prompt template hyperparameter instead of the LLM you are iterating on when working with the answer relevancy metric. To be more specific, a low answer relevnacy score signifies that you need to improve examples used in prompt templates for better in-context learning, or include more fine-grained prompting for better instruction following capabilities to generate more relevant responses.
    * Faithfulness: the faithfulness metric determines how much the answers generated by your RAG generator are hallucinations. This concerns the LLM hyperparameter, and you'll want to switch to a different LLM or even fine-tune your own if your LLM is unable to leverage the retrieval context supplied to it to generate grounded answers.
    * Contextual Relevancy: the contextual relevancy metric determines whether the text chunks retrieved by your RAG retriever are relevant to producing the ideal answer for a user input. This concerns the chunk size, top-K and embedding model hyperparameter. A good embedding model ensures you're able to retrieve text chunks that are semantically similar to the embedded user query, while a good combination of chunk size and top-K ensures you only select the most important bits of information in your knowledge base.

In [27]:
# generate answers form current rag pipeline without re-ranker
# make testcases and put it them in list
test_cases = []
for question in questions_answers:
    # input question 
    input_question = question["question"]
    # query using query engine
    actual_output = query_engine.query(input_question)
    # there is a think part so remove that think part
    cleaned_actual_output = re.sub(r'<think>.*?</think>', '', actual_output.response, flags=re.DOTALL).strip()

    # then get the retrieved context from metadata dictionary 
    retrieved_context_dict = [dict_val for dict_val in actual_output.metadata.values()]
    # get the window value that is the context
    retrieved_context = [text['window'] for text in retrieved_context_dict]
    # join the retrieved nodes
    retrieved_text = "\n".join(retrieved_context)

    # make the test case 
    test_case = LLMTestCase(input=input_question, actual_output=cleaned_actual_output, retrieval_context=[retrieved_text], expected_output = question["expected_answer"])

    # add to the list 
    test_cases.append(test_case)

In [28]:
# convert test cases to dictionaries
test_cases_dict = [test_case.__dict__ for test_case in test_cases]

# save to JSON file
with open("test_cases.json", "w") as f:
    json.dump(test_cases_dict, f, indent=4)

print("Test cases saved successfully!")

Test cases saved successfully!


In [25]:
# load test cases from JSON
with open("test_cases.json", "r") as f:
    test_cases_data = json.load(f)

# convert dictionaries back to LLMTestCase objects
loaded_test_cases = [LLMTestCase(**data) for data in test_cases_data]

print("Test cases loaded successfully!")

Test cases loaded successfully!


In [26]:
pprint(loaded_test_cases[:5])

[LLMTestCase(input='What is TRONIC.LK?',
             actual_output='TRONIC.LK is a leading electronic component and '
                           'module sourcing company based in Sri Lanka, '
                           'established under Sigma Electronics (Pvt) Ltd. '
                           'They supply components such as Arduinos, Raspberry '
                           'Pis, Orange Pis, Micro:bits, NodeMCUs, Atmel & '
                           'Microchip microcontrollers, ICs & passive '
                           'components, Creality 3D Printers & filaments, '
                           'Pneumatic parts, CNC accessories, Inverters, '
                           'Battery chargers, Multimeters, and Ronix Tools. '
                           'The company is known for its high-quality products '
                           'and has been operational since 2011, operating '
                           '24/7 from 9 AM to 7 PM, excluding holidays. They '
                           'provid

In [54]:
# make objects on relevant metrics
answer_relevancy = AnswerRelevancyMetric()
faithfulness = FaithfulnessMetric()
contextual_relevancy = ContextualRelevancyMetric()

* before run the test set the local model in the terminal to deep-eval to as evaluate model   
    -   'deepeval set-ollama deepseek-r1:1.5b'

In [ ]:
evaluate(test_cases=test_cases, metrics=[answer_relevancy, faithfulness, contextual_relevancy], identifier="v1.1")

✨ You're running DeepEval's latest Answer Relevancy Metric! (using local model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using local model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using local model, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 01:58, 118.66s/test case]




Metrics Summary

  - ✅ Answer Relevancy (score: 0.6666666666666666, threshold: 0.5, strict: False, evaluation model: local model, reason: This statement is about TRONIC.LK's services or support, not its identity., error: None)
  - ✅ Faithfulness (score: 0.8, threshold: 0.5, strict: False, evaluation model: local model, reason: The Faithfulness Score of 0.80 reflects a significant contradiction in the actual output, as TRONIC.LK is incorrectly identified as part of Sigma Electronics (Pvt) Ltd., which does not align with the retrieval context., error: None)
  - ✅ Contextual Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: local model, reason: TRONIC.LK was introduced to the electronics industry in Sri Lanka in 2011, specifically introducing Arduino development boards and modules. Today, TRONIC.LK is one of the leading electronic stores in Sri Lanka, and they supply a wide range of components including Arduinos, Raspberry Pis, and other hardware products. Their hi

✓ Tests finished 🎉! View results on 
]8;id=242754;https://app.confident-ai.com/project/cm7exqe4a0bt413kajni6zycg/evaluation/test-runs/cm7xm39e3107cpbig8un7ki3k/test-cases\https://app.confident-ai.com/project/cm7exqe4a0bt413kajni6zycg/evaluation/test-runs/cm7xm39e3107cpbig8un7ki3k/test-]8;;\
]8;id=242754;https://app.confident-ai.com/project/cm7exqe4a0bt413kajni6zycg/evaluation/test-runs/cm7xm39e3107cpbig8un7ki3k/test-cases\cases]8;;\.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.6666666666666666, reason="This statement is about TRONIC.LK's services or support, not its identity.", strict_mode=False, evaluation_model='local model', error=None, evaluation_cost=0.0, verbose_logs='Statements:\n[\n    "TRONIC.LK is a leading electronic component and module sourcing company based in Sri Lanka.",\n    "They offer a wide range of components including Arduinos, Raspberry Pis, Orange Pis, etc.",\n    "They have an email system at check@tronic.lk for item availability or pricing."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "no",\n        "reason": "This statement is about TRONIC.LK\'s services or support, not its identity."\n    }\n]'), MetricData(name='Faithfulness', threshold

In [ ]:
evaluate(test_cases=test_cases[:5], metrics=[answer_relevancy, faithfulness, contextual_relevancy], identifier="v1.1")